In [1]:
import os
# os.chdir('../py')
import gc
import sys
import warnings
warnings.filterwarnings('ignore')
from copy import deepcopy
from glob import glob

import feather
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams['font.family'] = 'YuGothic'
sns.set(style="whitegrid", font='YuGothic', palette="muted",
        color_codes=True, rc={'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

In [2]:
#前処理
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [3]:

#各種機械学習モデル
#LogisticRegression
from sklearn.linear_model import LogisticRegression
#Linear Support Vector Machine
from sklearn.svm import LinearSVC
#Support Vector Machine
from sklearn.svm import SVC
#決定木
from sklearn.tree import DecisionTreeClassifier
#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
#k近傍方（k-neighbor）
from sklearn.neighbors import KNeighborsClassifier

#学習方法（GridSearchCV）
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

#scikiti-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,GroupKFold
from sklearn.metrics import roc_auc_score

#スコアリング（f値）
from sklearn.metrics import f1_score

#lightgbm
import lightgbm as lgb
import pandas_profiling as pdp

/Users/yuki.matsumoto/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [17]:
df = pd.DataFrame([
        ['S', 100, 'cola'],
        ['M', 150, 'tea'],
        ['L', 200, 'coffee']])
df.columns = ['size', 'price', 'label']
df

,size,price,label
0,S,100,cola
1,M,150,tea
2,L,200,coffee


In [18]:
size_mapping = {'L': 3, 'M': 2, 'S': 1}
df['size'] = df['size'].map(size_mapping)
df

,size,price,label
0,1,100,cola
1,2,150,tea
2,3,200,coffee


In [19]:
#LabelEncoderのインスタンスを生成
le = LabelEncoder()
#ラベルを覚えさせる
le = le.fit(df['label'])
#ラベルを整数に変換
df['label'] = le.transform(df['label'])
df

,size,price,label
0,1,100,1
1,2,150,2
2,3,200,0


In [20]:
#one-hot エンコーダの生成
oe = OneHotEncoder(categorical_features=[2])
#one-hot エンコーディングを実行
array = oe.fit_transform(df).toarray()
array

/Users/yuki.matsumoto/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[  0.,   1.,   0.,   1., 100.],
       [  0.,   0.,   1.,   2., 150.],
       [  1.,   0.,   0.,   3., 200.]])

In [21]:
#DataFrameに変換
columns = ['label_coffee','label_cola','label_tea','size','price']
df1 = pd.DataFrame(data = array, columns = columns)
#列入れ替え
df1 = df1[['size','price','label_cola','label_tea','label_coffee']]
df1

,size,price,label_cola,label_tea,label_coffee
0,1.0,100.0,1.0,0.0,0.0
1,2.0,150.0,0.0,1.0,0.0
2,3.0,200.0,0.0,0.0,1.0


get_dummies

In [23]:
df['label'] = le.inverse_transform(df['label'])
df

,size,price,label
0,1,100,cola
1,2,150,tea
2,3,200,coffee


In [24]:
df_dummy = pd.get_dummies(df['label'])
df2 = pd.concat([df.drop(['label'],axis=1),df_dummy],axis=1)
df2

,size,price,coffee,cola,tea
0,1,100,0,1,0
1,2,150,0,0,1
2,3,200,1,0,0


In [26]:
df_dummy = pd.get_dummies(df['label'],drop_first = True)
df3 = pd.concat([df.drop(['label'],axis=1),df_dummy],axis=1)
df3

,size,price,cola,tea
0,1,100,1,0
1,2,150,0,1
2,3,200,0,0
